In [1]:
import os
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import networkx as nx
from networkx.algorithms.similarity import graph_edit_distance

In [2]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Graph Similarity Measure between nf_ToN_iot and nf_BoN_iot**

In [4]:
df_nf_ToN_iot_v1=pd.read_parquet('/content/gdrive/MyDrive/csci-e-599a-2024-cybersecurity/data/parquet/netflow/ip_port/NF-ToN-IoT_chunks')

In [5]:
df_nf_ToN_iot_v1[df_nf_ToN_iot_v1.duplicated()]

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
__null_dask_index__,,,,,,,,,,,,,


In [ ]:
# No Duplicates Found

In [6]:
df_nf_ToN_iot_v1.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
__null_dask_index__,,,,,,,,,,,,,
0,192.168.1.195,63318,52.139.250.253,443,6,91.00,181,165,2,1,24,327,0
1,192.168.1.79,57442,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0
2,192.168.1.79,57452,239.255.255.250,15600,17,0.00,63,0,1,0,0,0,0
3,192.168.1.193,138,192.168.1.255,138,17,10.16,472,0,2,0,0,0,0
4,192.168.1.79,51989,192.168.1.255,15600,17,0.00,63,0,1,0,0,0,0


In [7]:
df_nf_ToN_iot_v1.shape

(846951, 13)

In [8]:
df_nf_BoT_iot_v1=pd.read_parquet('/content/gdrive/MyDrive/csci-e-599a-2024-cybersecurity/data/parquet/netflow/ip_port/NF-BoT-IoT_chunks')

In [9]:
df_nf_BoT_iot_v1[df_nf_BoT_iot_v1.duplicated()]

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
__null_dask_index__,,,,,,,,,,,,,


**No Duplicates Found**

In [10]:
df_nf_BoT_iot_v1.shape

(510941, 13)

In [11]:
df_nf_BoT_iot_v1.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label'],
      dtype='object')

In [12]:
df_nf_BoT_iot_v1=df_nf_BoT_iot_v1[['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS']]

In [14]:
#Excluding labels for df_nf_BoT_iot_v1

In [13]:
df_nf_ToN_iot_v1.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label'],
      dtype='object')

In [15]:
#Excluding labels for df_nf_ToN_iot_v1

In [14]:
df_nf_ToN_iot_v1=df_nf_ToN_iot_v1[['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS']]

In [15]:
G1 = nx.from_pandas_edgelist(df_nf_ToN_iot_v1, 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR', create_using=nx.DiGraph(),edge_attr=True)

In [16]:
G2 = nx.from_pandas_edgelist(df_nf_BoT_iot_v1, 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR', create_using=nx.DiGraph(),edge_attr=True)

**Graph Edit Distance**

In [ ]:
#ged = graph_edit_distance(G1, G2, edge_match=lambda x, y: x.get('PROTOCOL') == y.get('PROTOCOL') and x.get('L7_PROTO') == y.get('L7_PROTO')
 #                         and x.get('IN_BYTES')==y.get('IN_BYTES') and x.get('OUT_BYTES')==y.get('OUT_BYTES') and x.get('IN_PKTS')==y.get('IN_PKTS')
  #                        and x.get('OUT_PKTS')==y.get('OUT_PKTS') and x.get('TCP_FLAGS')==y.get('TCP_FLAGS') and x.get('FLOW_DURATION_MILLISECONDS')==y.get('FLOW_DURATION_MILLISECONDS')
   #                       )

#print("Graph Edit Distance:", ged)

Failing due to Memory Issue

**Finding Cosine Similarity for Embeddings Generated from Node2Vec Between Two Graph Set**


1.   df_nf_ToN_iot_v1 and df_nf_BoT_iot_v1
2.   df_nf_ToN_iot_v1 and df_nf_UNSW_NB15





In [17]:
import networkx as nx
from node2vec import Node2Vec
from sklearn.metrics.pairwise import cosine_similarity


# Train edge2vec model for graph 1
node2vec_model1 = Node2Vec(G1, dimensions=64, walk_length=30, num_walks=200, workers=4)
embeddings1 = node2vec_model1.fit().wv.vectors

# Train edge2vec model for graph 2
node2vec_model2 = Node2Vec(G2, dimensions=64, walk_length=30, num_walks=200, workers=4)
embeddings2 = node2vec_model2.fit().wv.vectors


# Reshape embeddings for cosine similarity calculation
embeddings1 = embeddings1.reshape(-1, embeddings1.shape[1])
embeddings2 = embeddings2.reshape(-1, embeddings2.shape[1])

print(embeddings1.shape)
print(embeddings2.shape)

# Calculate cosine similarity
cosine_sim = cosine_similarity(embeddings1, embeddings2)
average_similarity = np.mean(cosine_sim)


#print(embeddings1)

# Print the cosine similarity matrix
print(cosine_sim)

print("average_similarity",average_similarity)


Computing transition probabilities:   0%|          | 0/1501 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/131 [00:00<?, ?it/s]

(1501, 64)
(131, 64)
[[0.30662134 0.22747512 0.23078051 ... 0.16246511 0.23987928 0.2510439 ]
 [0.61705315 0.67723954 0.66560817 ... 0.18881083 0.64040947 0.6280927 ]
 [0.49562508 0.5045937  0.5455535  ... 0.01591767 0.47606227 0.50173104]
 ...
 [0.51602054 0.51761943 0.5501749  ... 0.05170628 0.49160337 0.5182792 ]
 [0.5247249  0.5370749  0.5653132  ... 0.03276088 0.5055238  0.5322301 ]
 [0.281765   0.25597477 0.25342417 ... 0.09214698 0.28548244 0.26964408]]
average_similarity 0.4827026


In [18]:
df_nf_UNSW_NB15=pd.read_parquet('/content/gdrive/MyDrive/csci-e-599a-2024-cybersecurity/data/parquet/netflow/ip_port/NF-UNSW-NB15_chunks')

In [19]:
df_nf_UNSW_NB15.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
__null_dask_index__,,,,,,,,,,,,,
0,149.171.126.0,62073,59.166.0.5,56082,6,0.0,9672,416,11,8,25,15,0
1,149.171.126.2,32284,59.166.0.5,1526,6,0.0,1776,104,6,2,25,0,0
2,149.171.126.0,21,59.166.0.1,21971,6,1.0,1842,1236,26,22,25,1111,0
3,59.166.0.1,23800,149.171.126.0,46893,6,0.0,528,8824,10,12,27,124,0
4,59.166.0.5,63062,149.171.126.2,21,6,1.0,1786,2340,32,34,25,1459,0


In [20]:
df_nf_UNSW_NB15.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label'],
      dtype='object')

In [21]:
df_nf_UNSW_NB15=df_nf_UNSW_NB15[['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS']]

In [22]:
G3 = nx.from_pandas_edgelist(df_nf_UNSW_NB15, 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR', create_using=nx.DiGraph(),edge_attr=True)

In [24]:
# Train edge2vec model for graph 1
node2vec_model1 = Node2Vec(G1, dimensions=64, walk_length=30, num_walks=200, workers=4)
embeddings1 = node2vec_model1.fit().wv.vectors

# Train edge2vec model for graph 2
node2vec_model2 = Node2Vec(G3, dimensions=64, walk_length=30, num_walks=200, workers=4)
embeddings2 = node2vec_model2.fit().wv.vectors


# Reshape embeddings for cosine similarity calculation
embeddings1 = embeddings1.reshape(-1, embeddings1.shape[1])
embeddings2 = embeddings2.reshape(-1, embeddings2.shape[1])

print(embeddings1.shape)
print(embeddings2.shape)

# Calculate cosine similarity
cosine_sim = cosine_similarity(embeddings1, embeddings2)
average_similarity = np.mean(cosine_sim)


#print(embeddings1)

# Print the cosine similarity matrix
print(cosine_sim.shape)

print("average_similarity",average_similarity)

Computing transition probabilities:   0%|          | 0/1501 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/44 [00:00<?, ?it/s]

(1501, 64)
(44, 64)
(1501, 44)
average_similarity 0.35102224
